In [5]:
import os
import pandas as pd

In [1]:
def load_dataset(file_path, file_name, nrows=10000000):
    full_path = os.path.join(file_path, file_name)
    data_set = pd.read_csv(full_path, header=0, nrows=nrows, dtype={'Column85': str}, low_memory=False)
    
    data_set.columns = data_set.columns.str.strip()
    
    return data_set

def get_samples_by_label(data_frame, label):    
    filtered_samples = data_frame[data_frame["Label"] == label]
    
    return filtered_samples

def combine_datasets(file_path, file_names, nrows=15000):
    
    dataframes = []
    
    for name in file_names:
        try:
            df = load_dataset(file_path, name, nrows)
            dataframes.append(df)
        except Exception as e:
            print(f"Error reading {os.path.join(file_path, name)}: {e}")
    
    combined_df = pd.concat(dataframes, ignore_index=True)
    
    return combined_df


def calculate_label_distribution(dataframe, label_column):
    total_samples = dataframe.shape[0]
    label_counts = dataframe[label_column].value_counts()
    
    distribution = {}
    for label, count in label_counts.items():
        percentage = (count / total_samples) * 100
        distribution[label] = {
            'count': count,
            'percentage': percentage
        }
        
    return distribution

def display_label_distribution(label_distribution):
    for label, stats in label_distribution.items():
        print(f"Samples Count: {label}: {stats['count']}")
        print(f"Percentage distribution of data: {label}: {stats['percentage']:.2f}%")
    

def display_unique_labels(data_frame):
    unique_labels = data_frame["Label"].unique()
    print("Unique Labels:", unique_labels)

def display_dataset_length(data_frame):
    length = len(data_frame)
    print("Dataset length:", length)